<a href="https://colab.research.google.com/github/deepakmoud/Signature-Verification/blob/master/Signature_Recognition_VGG19_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing the Keras libraries and packages
import tensorflow as tf
from tensorflow.python import keras
from keras.layers import Input, Flatten, Dense
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model

from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications import Xception
from keras.applications import VGG19
import warnings 
warnings.filterwarnings('ignore')
img_width, img_height = 224, 224

Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [23]:
# Input parameter
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test'
TEST_DIR='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/TEST1/'
filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/vgg16_GDPS_xfer_weights.hdf5'
nb_epochs = 50
SIGNATURE_CLASSES = ['001', '002', '003','004','006','009','012','013','014','015','016','017','018','019','020','021','022','023','024','025','026','027','028','029','030','031','032','033','034','035','036','037','038','039','040','041','042','043','044','045','046','047','048','049','050','051','052','053','054','055','056','057','058','059','060','061','062','063','064','065','066','067','068','069']
print("Input parameters are assigned")

Input parameters are assigned


In [4]:
# image data generation
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
val_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/'
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True, class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(val_data_dir, target_size=(img_width, img_height), 
                                                    batch_size=32,shuffle=True,class_mode='categorical')

Found 672 images belonging to 64 classes.
Found 224 images belonging to 64 classes.


In [7]:
#Load the pretrained Network
vgg19_model = VGG19( 
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(img_height,img_width,3),
    pooling=None,
    classes=1000
    
)
print("pretrained Network is loaded")

pretrained Network is loaded


In [8]:
vgg19_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [9]:
# Freeze the layers
for layer in vgg19_model.layers:
    layer.trainable = False
print("Pretrained layers are freezed")

Pretrained layers are freezed


In [13]:
# Model Creation
model = Sequential()
model.add(vgg19_model)
#add fully connected layer:
input_layer=model.add(Flatten())
hidden_layer=model.add(Dense(128, activation='relu', name='hidden_layer'))
classification_layer=model.add(Dense(64, activation='relu', name='classification_layer'))
output_layer=model.add(Dense(64, activation='softmax', name='output_layer')) 
print("All layers top of pretrained layers are developed")

All layers top of pretrained layers are developed


In [14]:
# Model compilation
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [15]:
# model weights saving after each epoch
checkpoint = ModelCheckpoint(filepath = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/test/weights-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
print('Check point is created')

Check point is created


In [16]:
history = model.fit_generator( train_generator, callbacks = callbacks_list, nb_epoch=nb_epochs, validation_data=validation_generator)
print('Training Completed!')

Epoch 1/50
21/21 [==============================] - 242s 12s/step - loss: 4.2108 - accuracy: 0.0536 - val_loss: 3.8482 - val_accuracy: 0.1071
Epoch 2/50
21/21 [==============================] - 10s 480ms/step - loss: 3.4058 - accuracy: 0.2232 - val_loss: 3.0670 - val_accuracy: 0.3527
Epoch 3/50
21/21 [==============================] - 10s 487ms/step - loss: 2.5692 - accuracy: 0.4256 - val_loss: 2.2012 - val_accuracy: 0.4598
Epoch 4/50
21/21 [==============================] - 10s 473ms/step - loss: 1.7016 - accuracy: 0.6205 - val_loss: 1.7809 - val_accuracy: 0.7054
Epoch 5/50
21/21 [==============================] - 10s 476ms/step - loss: 0.9990 - accuracy: 0.7932 - val_loss: 0.6212 - val_accuracy: 0.8125
Epoch 6/50
21/21 [==============================] - 10s 470ms/step - loss: 0.5897 - accuracy: 0.8973 - val_loss: 1.0127 - val_accuracy: 0.8661
Epoch 7/50
21/21 [==============================] - 10s 473ms/step - loss: 0.3635 - accuracy: 0.9628 - val_loss: 0.3758 - val_accuracy: 0.9018


In [42]:
# save model and architecture to single file
model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_vgg19_model.h5')
model.summary()

print("Saved model to disk")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 23,248,192
Trainable params: 3,223,808
Non-trainable params: 20,024,384
_________________________________________________________________
Saved model to disk


In [43]:
# Loading saved model from Drive.
from keras.models import load_model
model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_vgg19_model.h5')
print("Model is Loaded")
model.summary()

Model is Loaded
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
_________________________________________________________________
output_layer (Dense)         (None, 64)                4160      
Total params: 23,248,192
Trainable params: 3,223,808
Non-trainable params: 20,024,384
_________________________________________________________________


In [0]:
import os
train_data_dir ='/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/train/'
def get_images(fish):
    """Load files from train folder"""
    fish_dir = train_data_dir+'{}'.format(fish)
    images = [fish+'/'+im for im in os.listdir(fish_dir)]
    return images

In [45]:
pip install scipy==1.1.0

In [0]:
def read_image(src):
    import os
    from scipy import misc
    filepath=src
    im=misc.imread(filepath)
    import scipy.misc  as mc
     
    return mc.imresize(im,(img_width, img_height))

In [47]:
files = []
y_all = []
for fish in SIGNATURE_CLASSES:
    fish_files = get_images(fish)
    files.extend(fish_files)
    
    y_fish = np.tile(fish, len(fish_files))
    y_all.extend(y_fish)
    print("{0} photos of {1}".format(len(fish_files), fish))
    
y_all = np.array(y_all)
print(len(files))
print(len(y_all))


19 photos of 001
19 photos of 002
18 photos of 003
18 photos of 004
18 photos of 006
18 photos of 009
18 photos of 012
9 photos of 013
18 photos of 014
18 photos of 015
18 photos of 016
9 photos of 017
9 photos of 018
9 photos of 019
9 photos of 020
9 photos of 021
9 photos of 022
9 photos of 023
9 photos of 024
9 photos of 025
9 photos of 026
9 photos of 027
9 photos of 028
9 photos of 029
9 photos of 030
9 photos of 031
9 photos of 032
9 photos of 033
9 photos of 034
9 photos of 035
9 photos of 036
9 photos of 037
9 photos of 038
9 photos of 039
9 photos of 040
9 photos of 041
9 photos of 042
9 photos of 043
9 photos of 044
9 photos of 045
9 photos of 046
9 photos of 047
6 photos of 048
9 photos of 049
9 photos of 050
9 photos of 051
9 photos of 052
9 photos of 053
9 photos of 054
9 photos of 055
9 photos of 056
9 photos of 057
9 photos of 058
9 photos of 059
9 photos of 060
9 photos of 061
9 photos of 062
9 photos of 063
9 photos of 064
9 photos of 065
9 photos of 066
12 photos of 0

In [48]:
X_all = np.ndarray((len(files),img_width, img_height , 3), dtype=np.uint8)
print(X_all.shape)
for i, im in enumerate(files): 
    X_all[i] = read_image(train_data_dir+im)
    print(X_all[i])   
   
    if i%1000 == 0: print('Processed {} of {}'.format(i, len(files)))


print(X_all.shape)

Streaming output truncated to the last 5000 lines.
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 252]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]]
[[[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 [[245 253 253]
  [245 253 253]
  [245 253 253]
  ...
  [245 253 253]
  [245 253 253]
  [245 253 253]]

 ...

 

In [49]:
import os
test_files = [im for im in os.listdir(TEST_DIR)]
test = np.ndarray((len(test_files), img_width, img_height, 3), dtype=np.uint8)

for i, im in enumerate(test_files): 
    test[i] = read_image(TEST_DIR+im)
    
test_preds = model.predict(test, verbose=1)
submission = pd.DataFrame(test_preds, columns=SIGNATURE_CLASSES)
submission.insert(0, 'image', test_files)
submission.head()

submission.to_csv('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureResultsvgg19.csv',index=False)

12/12 [==============================] - 0s 7ms/step


In [50]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
from keras.models import load_model
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Click to know Name " )
output = widgets.Output()

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_VGG16_model.h5')
img_path = '/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/11_052.png'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

block4_pool_features = model.predict(x)
print(block4_pool_features)
label_index=block4_pool_features.argmax()
print(label_index)
def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Signature is of  "  +SIGNATURE_CLASSES[label_index])

button.on_click(on_button_clicked)
display(button, output)

[[1.07392574e-10 2.33686183e-15 1.57053474e-16 3.63359703e-07
  2.06278397e-12 5.14659648e-10 5.26925413e-11 1.13121491e-10
  1.23408617e-09 1.19378711e-07 1.47738384e-11 7.59146468e-10
  1.12326743e-08 1.42529743e-07 1.41109938e-14 2.81836261e-08
  1.50759927e-09 1.66780492e-10 1.15319144e-05 1.17536871e-07
  1.68616805e-06 4.35485390e-06 1.82785828e-10 6.03519226e-08
  2.88963997e-06 2.22058858e-07 1.76758001e-11 3.87884948e-14
  1.33578495e-11 8.05362808e-12 1.53551737e-05 4.03263585e-17
  1.60578281e-06 3.29200274e-12 6.05249878e-11 7.52718188e-06
  2.23652208e-07 2.73917353e-16 1.03515517e-13 6.31267824e-12
  2.15225684e-10 5.15619649e-06 1.58937974e-10 6.50012975e-13
  1.65133188e-05 1.07744859e-06 9.99741256e-01 3.34272526e-10
  2.03904551e-06 1.76026403e-13 4.70588737e-13 1.54898361e-09
  1.12556258e-15 3.14181463e-08 1.81714000e-04 1.67796188e-09
  8.04147200e-12 1.23990727e-12 3.76783369e-08 7.23589721e-07
  1.44527593e-11 2.06035700e-09 3.21946949e-08 5.13825989e-06]]
46


Button(description='Click to know Name ', style=ButtonStyle())

Output()

In [51]:
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_vgg19_model.h5')
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(validation_generator)
print('test loss, test acc:', results)


# Evaluate on test data
7/7 [==============================] - 1s 183ms/step
test loss, test acc: [0.00649852491915226, 0.9330357313156128]


In [52]:
# Extracting Features from classification Layer
from keras.models import Model
layer_name= 'classification_layer'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
print("Imtermediate model is constructed")


Imtermediate model is constructed


In [53]:
# Compilation of intermediate model
intermediate_layer_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print("Model is Complied")

Model is Complied


In [54]:
# Saving intermediate model
intermediate_layer_model.save('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_vgg19folder_Intermediatemodel.h5')
intermediate_layer_model.summary()

print("Saved Intermediate model to disk")

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg19 (Model)                (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               3211392   
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 23,244,032
Trainable params: 3,219,648
Non-trainable params: 20,024,384
_________________________________________________________________
Saved Intermediate model to disk


In [55]:
# Loading Intermediate Model
from keras.models import load_model

model = load_model('/content/drive/My Drive/Dataset2/sign_data.zip (Unzipped Files)/sign_data/signatureRecognition_vgg19folder_Intermediatemodel.h5')
print("Intermediate model is loaded")

Intermediate model is loaded


In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception_input (InputLayer)  (None, 224, 224, 3)       0         
_________________________________________________________________
xception (Model)             (None, 7, 7, 2048)        20861480  
_________________________________________________________________
flatten_2 (Flatten)          (None, 100352)            0         
_________________________________________________________________
hidden_layer (Dense)         (None, 128)               12845184  
_________________________________________________________________
classification_layer (Dense) (None, 64)                8256      
Total params: 33,714,920
Trainable params: 12,853,440
Non-trainable params: 20,861,480
_________________________________________________________________


In [56]:
# Training Label feature identification(y_train)
import numpy as np
batch_size=32
sample_count=672
features = np.zeros(shape=(672, 64))  # Must be equal to the output of the convolutional base
labels = np.zeros(shape=(672, 64))
i = 0
for inputs_batch, labels_batch in train_generator:
  features_batch = model.predict(inputs_batch)
  features[i * batch_size: (i + 1) * batch_size] = features_batch
  labels[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels.shape)
print(features.shape)


(672, 64)
(672, 64)


In [57]:
 print(labels)

[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [58]:
#identification of training Labels
features_train=features
print(features_train.shape)
labels_train=labels
print(labels_train.shape)
print(labels)
#print(features_train)


(672, 64)
(672, 64)
[[0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [59]:
# identification of test labels
import numpy as np
batch_size=32
sample_count=224
features_test = np.zeros(shape=(224, 64))  # Must be equal to the output of the convolutional base
labels_test = np.zeros(shape=(224,64))
i = 0
for inputs_batch, labels_batch in validation_generator:
  features_batch = model.predict(inputs_batch)
  features_test[i * batch_size: (i + 1) * batch_size] = features_batch
  labels_test[i * batch_size: (i + 1) * batch_size] = labels_batch
  i += 1
  if i*batch_size  >= sample_count:
    break
print(labels_test.shape)
print(features_test.shape)
print(labels_test)

(224, 64)
(224, 64)
[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [60]:
# #identification of testing Labels
print(features_test.shape)
#labels_test=np.expand_dims(labels_test, axis=1)
print(labels_test.shape)
print(features_test)

(224, 64)
(224, 64)
[[ 0.          0.         13.80913353 ...  1.84460938  2.44026756
   0.        ]
 [ 0.          0.          2.53990698 ...  0.         18.94135666
   0.        ]
 [ 0.          0.          3.73219681 ...  1.17361534 17.37209892
   0.        ]
 ...
 [ 0.          0.          5.53793812 ...  1.80111492 18.8729248
   2.92887926]
 [ 0.          0.          4.59933186 ...  2.96519685 18.77487183
   0.70608896]
 [ 0.          0.         13.51272106 ...  0.         19.79763794
   1.11928606]]


In [61]:
print(labels_test)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [62]:
# SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
print(X_train.shape)
print(labels_train.shape)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
# Fitting SVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear', random_state = 0)
classifier.fit(X,y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

(672, 64)
(672, 64)


ValueError: ignored

In [63]:
# Kernel SVM
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='binary')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='binary')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='binary')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [64]:
# Random Forest
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 60.714
Precision: 97.143
Recall: 60.714
F-Measure: 74.725


In [65]:
# Decision Tree
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 73.214
Precision: 73.214
Recall: 73.214
F-Measure: 73.214


In [66]:
# Naive Bayes
# Feature Scaling
from sklearn.preprocessing import StandardScaler
from sklearn.utils import check_X_y
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
X, y = check_X_y(X_train, labels_train, accept_sparse=False, accept_large_sparse=True, dtype='numeric', order=None, copy=False, force_all_finite=True, ensure_2d=True, allow_nd=False, multi_output=True, ensure_min_samples=1, ensure_min_features=1, y_numeric=True, estimator=None)
# Fitting KernelSVM to the Training set
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X, y)
# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored

In [67]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)

# Fitting KernelSVM to the Training set
from sklearn.neighbors import KNeighborsClassifier
classifier =  KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier.fit(X_train, labels_train)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
#cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
#print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

Confusion matrix: 
Accuracy: 91.964
Precision: 94.064
Recall: 91.964
F-Measure: 93.002


In [68]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state = 0)
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(features_train)
X_test = sc.transform(features_test)
from sklearn.utils import check_X_y
X, y = check_X_y(
            X_train, labels_train, multi_output=True)
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X, y)

# Predicting the Test set results
y_pred = classifier.predict(X_test)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(labels_test, y_pred)
print('Confusion matrix: ')
print(cm)
# calculate Accuracy
from sklearn.metrics import accuracy_score
print('Accuracy: %.3f' % (accuracy_score(labels_test, y_pred)*100))
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# calculate precision
# Precision = TruePositives / (TruePositives + FalsePositives)
precision = precision_score(labels_test, y_pred, average='micro')
print('Precision: %.3f' % (precision*100))
# calculate recall
# Recall = TruePositives / (TruePositives + FalseNegatives)
recall = recall_score(labels_test, y_pred, average='micro')
print('Recall: %.3f' % (recall*100))
# F-Measure = (2 * Precision * Recall) / (Precision + Recall)
# calculate score
score = f1_score(labels_test, y_pred, average='micro')
print('F-Measure: %.3f' % (score*100))

ValueError: ignored